# Economic Data Analysis with Fred & Pandas
Pull The data, Visualize, and personalized.


<img src="https://datascientest.com/en/wp-content/uploads/sites/9/2022/01/illu_pandas-82-768x422.png" width="500" height="200" />



In [ ]:
!pip install fredapi > /dev/null
!pip install plotly


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import time
import seaborn as sns


plt.style.use('seaborn')
sns.set(style="whitegrid")
color_pal = sns.color_palette("husl")
pd.set_option('max_columns', 500)

!pip install fredapi
!pip install virtualenv

from fredapi import Fred

fred_key = 'e287d188ce48b837b565b5618cd6467a'

from kaggle_secrets import UserSecretsClient
secrets = UserSecretsClient()
fred_key = secrets.get_secret('fred-api')




# 1.Fred Object

In [ ]:
fred = Fred(api_key=fred_key)

# 2. Search for the data!

In [ ]:
sp_search = fred.search('S&P', order_by='popularity')

In [ ]:
sp_search.head()

# 3. Pull Raw Data & Plot

In [ ]:
import plotly.express as px
from fredapi import Fred

# Assuming you have already obtained your FRED API key
fred = Fred(api_key="e287d188ce48b837b565b5618cd6467a")

sp500 = fred.get_series(series_id='SP500')

# Created an interactive line plot
fig = px.line(x=sp500.index, y=sp500, labels={'y': 'S&P 500 Value'}, title='S&P 500 Performance Over Time')

# Updated layout for better readability
fig.update_layout(
    xaxis_title='Date',
    yaxis_title='S&P 500 Value',
    legend_title='Index',
    hovermode="x unified",  # Combine tooltips for all data points on the same x-axis value
    template="plotly_dark"   # Dark mode for better visualization
)

# Displaying the interactive plot
fig.show()


# 4. Pull and Join Multiple Data Series

In [ ]:
unemp_df = fred.search('unemployment rate state', filter=('frequency','Monthly'))
unemp_df = unemp_df.query('seasonal_adjustment == "Seasonally Adjusted" and units == "Percent"')
unemp_df = unemp_df.loc[unemp_df['title'].str.contains('Unemployment Rate')]

In [ ]:
all_results = []

for myid in unemp_df.index:
    results = fred.get_series(myid)
    results = results.to_frame(name=myid)
    all_results.append(results)
    time.sleep(0.1) # Don't request to fast and get blocked
uemp_results = pd.concat(all_results, axis=1)

In [ ]:
cols_to_drop = []
for i in uemp_results:
    if len(i) > 4:
        cols_to_drop.append(i)
uemp_results = uemp_results.drop(columns = cols_to_drop, axis=1)
print(uemp_results.head())


In [ ]:
uemp_states = uemp_results.copy()  #.drop('UNRATE', axis=1)
uemp_states = uemp_states.dropna()
id_to_state = unemp_df['title'].str.replace('Unemployment Rate in ','').to_dict()
uemp_states.columns = [id_to_state[c] for c in uemp_states.columns]

In [ ]:
uemp_states.plot(figsize=(12, 10))
plt.title('Unemployment Rates in Different States')
plt.xlabel('Time')
plt.ylabel('Unemployment Rate')
plt.tight_layout()
plt.show()




In [ ]:
# Plot States Unemployment Rate
px.line(uemp_states)

# Pull April 2020 Unemployment Rate Per State

In [ ]:
ax = uemp_states.loc[uemp_states.index == '2020-05-01'].T \
    .sort_values('2020-05-01') \
    .plot(kind='barh', figsize=(8, 12), width=0.7, edgecolor='black',
          title='Unemployment Rate by State, May 2020')
ax.legend().remove()
ax.set_xlabel('% Unemployed')
plt.show()

# Pull Participation Rate

In [ ]:
part_df = fred.search('participation rate state', filter=('frequency','Monthly'))
part_df = part_df.query('seasonal_adjustment == "Seasonally Adjusted" and units == "Percent"')

In [ ]:
part_id_to_state = part_df['title'].str.replace('Labor Force Participation Rate for ','').to_dict()

all_results = []

for myid in part_df.index:
    results = fred.get_series(myid)
    results = results.to_frame(name=myid)
    all_results.append(results)
    time.sleep(0.1) # Don't request to fast and get blocked
part_states = pd.concat(all_results, axis=1)
part_states.columns = [part_id_to_state[c] for c in part_states.columns]

# Plot Unemployment vs Participation for each state!

In [ ]:
# Fix DC
uemp_states = uemp_states.rename(columns={'the District of Columbia':'District Of Columbia'})

In [ ]:
fig, axs = plt.subplots(10, 5, figsize=(30, 30), sharex=True)
axs = axs.flatten()

i = 0
for state in uemp_states.columns:
    if state in ["District Of Columbia","Puerto Rico"]:
        continue
    ax2 = axs[i].twinx()
    uemp_states.query('index >= 2020 and index < 2022')[state] \
        .plot(ax=axs[i], label='Unemployment')
    part_states.query('index >= 2020 and index < 2022')[state] \
        .plot(ax=ax2, label='Participation', color=color_pal[1])
    ax2.grid(False)
    axs[i].set_title(state)
    i += 1
plt.tight_layout()
plt.show()

In [ ]:
state = 'California'
fig, ax = plt.subplots(figsize=(10, 5), sharex=True)
ax2 = ax.twinx()
uemp_states2 = uemp_states.asfreq('MS')
l1 = uemp_states2.query('index >= 2020 and index < 2022')[state] \
    .plot(ax=ax, label='Unemployment')
l2 = part_states.dropna().query('index >= 2020 and index < 2022')[state] \
    .plot(ax=ax2, label='Participation', color=color_pal[1])
ax2.grid(False)
ax.set_title(state)
fig.legend(labels=['Unemployment','Participation'])
plt.show()

In [ ]:
# Assuming 'state' and 'color_pal' are defined
state = 'Ohio'
color_pal = sns.color_palette("husl")

# Create a figure and axis
fig, ax = plt.subplots(figsize=(10, 5), sharex=True)
ax2 = ax.twinx()

# Set the frequency of the unemployment data
uemp_states2 = uemp_states.asfreq('MS')

# Plot the Unemployment data
l1 = uemp_states2.query('index >= 2020 and index < 2022')[state].plot(ax=ax, label='Unemployment')

# Plot the Participation data
l2 = part_states.dropna().query('index >= 2020 and index < 2022')[state] \
    .plot(ax=ax2, label='Participation', color=color_pal[1])

# Remove grid from the second axis
ax2.grid(False)

# Set title and labels
ax.set_title(state)
fig.legend(labels=['Unemployment', 'Participation'])

# Display the plot
plt.show()